In [1]:
import cv2
import skvideo.io
import numpy as np
import random
from collections import deque

/Users/mohan/anaconda3/lib/python3.5/site-packages/skvideo/__init__.py:356: UserWarning: avconv/avprobe not found in path: 
  warnings.warn("avconv/avprobe not found in path: " + str(path), UserWarning)


In [2]:
#used to generate rectangles of different coulrs to avoid confusion
def make_new_color():
    return [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]

In [3]:
#capturing video frames
#importing xml file to detect cars 
#referenced from https://github.com/andrewssobral/vehicle_detection_haarcascades
count = 0
cap = skvideo.io.vread('sample.mp4')
car_cascade = cv2.CascadeClassifier('cars.xml')

In [4]:
# i am tracking car using their centroids.so we get list of all centroids of all cars present in any given frame
#these centroids can be used to extend for collision detection.will try to extend based on the available time
centroids_list = deque([])
car_count = 0

with open("out.csv", "w") as o:
    pass

In [5]:
for i in range(0, cap.shape[0]):
    center1 = []
    center2 = []

    # convert to gray scale of each frames
    # print("i:", i)
    image = cap[i]
    gray_image = cv2.cvtColor(cap[i], cv2.COLOR_BGR2GRAY)

    # Detects cars of different sizes in the input image
    cars = car_cascade.detectMultiScale(gray_image, 1.1, 1)

    # loop over all the found cars
    for (x, y, w, h) in cars:
        xA = x
        xB = x + w
        yA = y
        yB = y + h
        # Enumerate over all the cars in centroids_list
        not_matched = True
        for idx, centroid_data in enumerate(centroids_list):
            if centroid_data[0] == count:
                continue
            if centroids_list[idx][4] == 0:
                centroids_list[idx][5] = "unlocked"
                centroids_list[idx][4] = 5

            # checking proximity using manhattan distance(indriectly calculates the diagonal distance between two points in successive frames)
            X = abs(float(centroid_data[2][0] + centroid_data[2][2]) / 2 - float(xA + xB) / 2)
            Y = abs(float(centroid_data[2][1] + centroid_data[2][3]) / 2 - float(yA + yB) / 2)
            # if there is a rectangle in 10 pixel proximity of a rectangle of previous frame then its assumed that
            # the car in the rectangle is same as it was in the previous frame
            # we can change this value to any other value based on the movement happening in the frames, if vehicles are moving
            # more than 10 pixels per frame then we change it to that value
            if X < 10 and Y < 10:

                not_matched = False
                centroids_list[idx][4] = 5
                centroids_list[idx][2] = [xA, yA, xB, yB]
                centroids_list[idx][6].append(np.sqrt(X ** 2 + Y ** 2) * 24)
                if centroids_list[idx][5] == "unlocked":

                    if centroids_list[idx][0] == count - 1:
                        centroids_list[idx][3] += 1

                    else:
                        centroids_list[idx][3] = 0

                if centroids_list[idx][3] == 3:
                    centroids_list[idx][5] = "locked"
                    centroids_list[idx][3] = 0
                cv2.rectangle(image, (xA, yA), (xB, yB), centroid_data[1], 2)
                cv2.putText(image, str(centroids_list[idx][6][-1]),
                            (xA, yA), cv2.FONT_HERSHEY_SIMPLEX, 0.5, centroid_data[1], 1, cv2.LINE_AA)
                centroids_list[idx][0] = count
                break

        #    If rectangle not matches with previous rectangles that means it is a new car so make a new rectangle
        if not_matched:
            color = make_new_color()

            # append new rectangle in previous cars list
            centroids_list.appendleft([count, color, (xA, yA, xB, yB), 1, 5, "unlocked", [0], car_count])
            car_count += 1
            cv2.rectangle(image, (xA, yA), (xB, yB), color, 2)
            cv2.putText(image, "0",
                        (xA, yA), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)
            prev_color = color
            prev_coords = [xA, yA, xB, yB]
            
    for idx, centroid_data in enumerate(centroids_list):

        if centroid_data[5] == "locked" and centroid_data[0] != count:
            centroids_list[idx][4] -= 1
            cv2.rectangle(image, (centroid_data[2][0], centroid_data[2][1]), (centroid_data[2][2], centroid_data[2][3]),
                          centroid_data[1], 2)
            cv2.putText(image, "0",
                        (centroid_data[2][0], centroid_data[2][1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, centroid_data[1], 1,
                        cv2.LINE_AA)
            if centroids_list[idx][4] == 0:
                centroids_list[idx][5] = "unlocked"
                centroids_list[idx][4] = 5
                centroids_list[idx][3] = 0

        if count - centroid_data[0] == 10:
            with open("out.csv", "a") as o:
                o.write(str(centroid_data[7]) + ": " + str(sum(centroid_data[6]) / len(centroid_data[6])) + "\n")

    centroids_list = deque([car_data for car_data in list(centroids_list) if count - car_data[0] < 10])

    # Display frames in a window
    cv2.imshow('video2', image)
    
    if cv2.waitKey(33) == 27:
        
        break
    # now you can see your images in the out folder in this directory, if there is no out folder make sure to make one
    cv2.imwrite("out/" + str(count) + ".jpg", image)
    count += 1


KeyboardInterrupt: 